In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install datatable

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 MB 9.5 MB/s eta 0:00:00


In [3]:
pip install pytorch_tabnet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 577.7 kB/s eta 0:00:00


In [4]:
import warnings

warnings.filterwarnings('ignore')

In [5]:
%reload_ext autoreload
%autoreload 2
import sys
sys.path.append("./drive/MyDrive/millennium_project")

from utils import *
import glob
import os, gc
import numpy as numpy
import pandas as pd
import scipy as sp
import datatable as dt
from collections import defaultdict
from tqdm.notebook import tqdm
from sklearn.utils import shuffle
from sklearn.metrics import r2_score
from numba import njit


from IPython.display import clear_output

from pytorch_tabnet.metrics import Metric
from pytorch_tabnet.tab_model import TabNetRegressor

import torch
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingWarmRestarts

import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

In [6]:
ticker_name = "GOOG"

In [7]:
N_FOLD = 5
N_MINS = 5
MIN_SIZE = 600 // N_MINS

SOL_NAME = ticker_name+'-tabnet'
DATA_NAME = ticker_name
mkdir(f'/content/drive/MyDrive/millennium_project/models/{SOL_NAME}/')

# Functions

In [8]:
def rmse_keras(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_true - y_pred)))


class RMSE(Metric):
    def __init__(self):
        self._name = "rmse"
        self._maximize = False

    def __call__(self, y_true, y_pred):
        y_true = y_true
        y_pred = y_pred
        return np.sqrt(np.mean(np.square(y_true - y_pred)))

def RMSELoss(y_pred, y_true):
    return torch.sqrt(torch.mean( ((y_true - y_pred)) ** 2 )).clone()

# Loading data

In [9]:
# train
df_train = dt.fread(f'./drive/MyDrive/millennium_project/dataset/train_{DATA_NAME}_LGB.csv').to_pandas()
fea_cols = fea_cols_GOOG
df_train["name"] = ticker_name
# result
df_result = dt.fread('./drive/MyDrive/millennium_project/dataset/train.csv').to_pandas()
df_result = gen_row_id(df_result)

In [10]:
df_train = gen_row_id(df_train)
df_train = add_time_fold(df_train, N_FOLD)

In [11]:
df_train.index = df_train.row_id
df_result.index = df_result.row_id

# Evaluation

In [12]:
tabnet_params = dict(
    # cat_idxs=cat_idxs,
    # cat_dims=cat_dims,
    # cat_emb_dim=1,
    n_d = 16,
    n_a = 16,
    n_steps = 2,
    gamma = 2,
    n_independent = 2,
    n_shared = 2,
    lambda_sparse = 0,
    optimizer_fn = Adam,
    optimizer_params = dict(lr = (2e-2)),
    mask_type = "entmax",
    scheduler_params = dict(T_0=200, T_mult=1, eta_min=1e-4, last_epoch=-1, verbose=False),
    scheduler_fn = CosineAnnealingWarmRestarts,
    seed = 42,
    verbose = 10

)
epochs = 1000

list_seeds = [11]

In [13]:
list_rmse = []
for i_seed, seed in enumerate(list_seeds):
    df_train = add_time_fold(df_train, N_FOLD, seed=seed)
    list_rmse += [[]]
    for i_fold in range(N_FOLD):
        gc.collect()
        df_tr = df_train.loc[df_train.fold!=i_fold]
        df_te = df_train.loc[df_train.fold==i_fold]

        X_train = df_tr[fea_cols].values
        y_train = df_tr[['value_to_predict']].values
        X_test = df_te[fea_cols].values
        y_test = df_te[['value_to_predict']].values
        idx_test = df_train.loc[df_train.fold==i_fold].index
        print(f'Fold {i_seed+1}/{len(list_seeds)} | {i_fold+1}/{N_FOLD}', X_train.shape, X_test.shape)


        # Callbacks
        ckp_path = f'./models/{SOL_NAME}/model_{i_seed}_{i_fold}.hdf5'

        model = TabNetRegressor(**tabnet_params)
        model.fit(X_train, y_train,
            eval_set=[(X_test, y_test)],
            max_epochs=10000,
            patience=50,
            batch_size=1024*20,
            virtual_batch_size=128*20,
            num_workers=8,
            drop_last=False,
            eval_metric=[RMSE],
            loss_fn=RMSELoss
        )

        y_pred = model.predict(X_test)
        curr_rmse = rmse_keras(y_test, y_pred)
        list_rmse[-1] += [curr_rmse]
        model.save_model(ckp_path)
        # generate and save preds
        df_result.loc[idx_test, f'pred_{i_seed}'] = y_pred
        clear_output()
        print(list_rmse)

[[<tf.Tensor: shape=(), dtype=float64, numpy=0.06467048349045863>, <tf.Tensor: shape=(), dtype=float64, numpy=0.05961865821586411>, <tf.Tensor: shape=(), dtype=float64, numpy=0.056665125486465295>, <tf.Tensor: shape=(), dtype=float64, numpy=0.4297998894320371>, <tf.Tensor: shape=(), dtype=float64, numpy=0.0622850216287197>]]


In [15]:
df_result.to_csv(f'./drive/MyDrive/millennium_project/results/{SOL_NAME}.csv', index=False)

In [17]:
for i in range(len(list_seeds)):
    print(i, rmse(df_result['value_to_predict'], df_result[f'pred_{i}']))
print('All: ', rmse(df_result['value_to_predict'], df_result[[f'pred_{i}' for i in range(len(list_seeds))]].mean(axis=1)))

0 0.19971895941210643
All:  0.19971895941210643


In [21]:
df_result = df_result.dropna()

In [22]:
for i in range(len(list_seeds)):
    print(i, rmse(df_result['value_to_predict'], df_result[f'pred_{i}']))
print('All: ', rmse(df_result['value_to_predict'], df_result[[f'pred_{i}' for i in range(len(list_seeds))]].mean(axis=1)))

0 0.1997189594121064
All:  0.1997189594121064
